## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Georgetown,MY,5.4112,100.3354,84.13,78,20,3.44,few clouds
1,1,Upernavik,GL,72.7868,-56.1549,37.56,84,100,11.14,overcast clouds
2,2,Port Alfred,ZA,-33.5906,26.8910,67.91,38,0,12.57,clear sky
3,3,Punta Arenas,CL,-53.1500,-70.9167,37.51,65,0,5.75,clear sky
4,4,San Cristobal,VE,7.7669,-72.2250,82.58,56,50,6.40,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you woule like for your trip?"))

What is the minimum temperature you would like for your trip?66
What is the maximum temperature you woule like for your trip?82


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferences_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) &
                                  (city_data_df["Max Temp"] <= max_temp)]
preferences_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Port Alfred,ZA,-33.5906,26.8910,67.91,38,0,12.57,clear sky
10,10,Nabire,ID,-3.3667,135.4833,74.37,95,79,3.87,broken clouds
11,11,Vaini,TO,-21.2000,-175.2000,73.56,94,75,3.44,broken clouds
12,12,Rikitea,PF,-23.1203,-134.9692,72.14,73,48,11.27,light rain
22,22,Saint-Philippe,RE,-21.3585,55.7679,69.39,75,91,14.92,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferences_df.isnull().values.any()

True

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferences_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Alfred,ZA,67.91,clear sky,-33.5906,26.8910,
10,Nabire,ID,74.37,broken clouds,-3.3667,135.4833,
11,Vaini,TO,73.56,broken clouds,-21.2000,-175.2000,
12,Rikitea,PF,72.14,light rain,-23.1203,-134.9692,
22,Saint-Philippe,RE,69.39,overcast clouds,-21.3585,55.7679,
30,Cape Town,ZA,66.88,clear sky,-33.9258,18.4232,
31,Quatre Cocos,MU,68.11,light rain,-20.2078,57.7625,
33,Bengkulu,ID,72.59,broken clouds,-3.8004,102.2655,
34,Beyneu,KZ,78.39,few clouds,45.3167,55.2000,
36,Kapaa,US,80.58,overcast clouds,22.0752,-159.3190,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in clean_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("not found, skip")
 
print("complete")
    # 6c. Get latitude and longitude from DataFrame

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    

    # 6e. Make request and retrieve the JSON data from the search. 
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        

not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
not found, skip
complete


In [9]:
hotel_df["Max Temp"] = hotel_df["Max Temp"].map("{:.1f}".format)

In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Port Alfred,ZA,67.9,clear sky,-33.5906,26.8910,The Halyards Hotel
10,Nabire,ID,74.4,broken clouds,-3.3667,135.4833,Hotel Nusantara I
11,Vaini,TO,73.6,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Rikitea,PF,72.1,light rain,-23.1203,-134.9692,People ThankYou
22,Saint-Philippe,RE,69.4,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [11]:
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [13]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
print(locations)
type(locations)

         Lat       Lng
2   -33.5906   26.8910
10   -3.3667  135.4833
11  -21.2000 -175.2000
12  -23.1203 -134.9692
22  -21.3585   55.7679
..       ...       ...
658   5.0314   25.1361
663   8.4596   25.6780
664  70.0311   20.9714
665   1.3346  124.8392
667  24.6333 -101.4167

[288 rows x 2 columns]


pandas.core.frame.DataFrame

In [15]:
# 11a. Add a marker layer for each city to the map. 
# locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             # max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))